Installing required libraries:

In [ ]:
%pip install boto3 nibabel numpy matplotlib scikit-image

Importing those libraries:

In [25]:
# Importing nessecary libaries:

import boto3
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import io
import tempfile
import os

1. Rendering Brain Slices
2. Data Preprocessing on the Brain Slices
3. Saving processed Brain Slices as '.png' Images to AWS S3 Buckets

In [ ]:
# Setting up the data pipeline to access the 5 brains in the AWS S3 Bucket folder path:

s3 = boto3.resource('s3')
bucket_name = 'chemocraft-data'
folder_path = 'MICCAI_BraTS2020_TrainingData/'
bucket = s3.Bucket(bucket_name)

# Setting Crop values for Data Preprocessing:

crop_left, crop_right = 20, 10
crop_top, crop_bottom = 30, 30

def render_nii_from_s3(filename): # Function to display the middle slice of each brain scan type
    print(f"Fetching file: {filename}")

    obj = bucket.Object(folder_path + filename)
    file_stream = io.BytesIO(obj.get()['Body'].read())

    # Using temp files for efficent computing:

    with tempfile.NamedTemporaryFile(suffix='.nii', delete=False) as temp_file:  # Disable auto-delete
        temp_file.write(file_stream.getvalue())
        temp_file.flush()

        temp_file_path = temp_file.name
        print(f"Temporary file created: {temp_file_path}")

    try:
        img = nib.load(temp_file_path)
        data = img.get_fdata() # Storing brain into data variable

        print(f"Data shape for {filename}: {data.shape}") # Displaying shape of each brain

        if data.size == 0: # if data is nonexistent
            print(f"No data found in {filename}")
            return

        slice_idx = data.shape[2] // 2 # Getting index of middle slice to display it

        plt.figure(figsize=(3, 3)) # Displaying in 3x3 square
        plt.imshow(data[:, :, slice_idx], cmap='gray') # Color is set to grayscale
        plt.title(f'{filename} - Slice {slice_idx}') # Creating a title for the image
        plt.axis('off')  # Hide axes for cleaner display
        plt.show() # Finally showing the image

    except Exception as e:
        print(f"Error loading file {filename}: {e}") # Reports problems with getting the file
    finally:
        try:
            os.remove(temp_file_path)  # We do not want to save files locally, so we now delete the temp files
            print(f"Deleted temporary file: {temp_file_path}")
        except OSError as cleanup_error:
            print(f"Error deleting temp file: {cleanup_error}")

def save_png_from_nii(filename): # Function which will save .png grayscale brain slices to AWS S3 Buckets
    print(f"Fetching file: {filename}")
    obj = bucket.Object(folder_path + filename)
    file_stream = io.BytesIO(obj.get()['Body'].read())

    with tempfile.NamedTemporaryFile(suffix='.nii', delete=False) as temp_file:  # Disable auto-delete
        temp_file.write(file_stream.getvalue())
        temp_file.flush()

        temp_file_path = temp_file.name
        print(f"Temporary file created: {temp_file_path}")
    
        try:
            img = nib.load(temp_file_path)
            data = img.get_fdata()
            
            start_y = crop_top
            end_y = data.shape[0] - crop_bottom
            start_x = crop_left
            end_x = data.shape[1] - crop_right

            if data.size == 0:
                print(f"No data found in {filename}")
                return
            
            for slice_idx in range(data.shape[2]): # For each slice of each brain
                slice_2d = data[:, :, slice_idx]
                cropped_slice = slice_2d[start_y:end_y, start_x:end_x]

                filename = filename.replace(".nii", "") # Removes the .nii part

                # Folder for each brain inside the Brain_Slices:

                brain_number = filename.split('_')[-2]
                scan_type = filename.split('_')[-1]

                slice_path = f"brain_slices/{brain_number}/{scan_type}"
                print(f"Saving file in directory: {slice_path}") 

                png_filename = f"{slice_path}/{slice_idx}.png" 
                
                with tempfile.NamedTemporaryFile(suffix= '.png', delete=False) as temp_png: # This part creates a temp png .file used to save the grayscale brain slice
                    mpimg.imsave(temp_png.name, cropped_slice, cmap='gray')
                    temp_png.flush()
                    temp_png.seek(0)
                    temp_png_name = temp_png.name
                try: 
                    s3.Bucket(bucket_name).upload_file(temp_png_name, f'tanmay/{png_filename}')
                    os.remove(temp_png_name)
                except Exception as e:
                    print(f"Error saving file: {png_filename}, {e}")
                    
        except Exception as e:
            print(f"Error saving file {filename}: {e}")

found_files = False

for obj in bucket.objects.filter(Prefix=folder_path):
    if obj.key.endswith('.nii'):
        found_files = True
        print(obj.key)
        filename = obj.key.split('/')[-2] + '/' + obj.key.split('/')[-1]  # Get the filename 
        print("     ")
        # render_nii_from_s3(filename)
        save_png_from_nii(filename)

if not found_files:
    print(f"No .nii files found in the folder {folder_path}")

Defining the GAN Architecture
1. The Generator
2. The Discriminator

The Generator:

In [12]:
dim = 100

In [ ]:
from tensorflow.keras import layers, models

def build_generator(latent_dim, output_shape):
    model = models.Sequential()
    model.add(layers.Dense(64 * 64 * 64, activation='relu', input_dim = latent_dim))
    model.add(layers.Reshape((64, 64, 64)))
    model.add(layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding="same", activation="relu"))
    model.add(layers.Conv2DTranspose(64, kernel_size=4, strides=2, padding="same", activation="relu"))
    model.add(layers.Conv2D(1, kernel_size=3, strides=1, padding="same", activation="tanh"))
    model.add(layers.Cropping2D(cropping=((3, 3), (3, 3))))
    return model

chemocraft_generator = build_generator(latent_dim=dim)
chemocraft_generator.summary()

# Generator's output shape should be the same shape as the cropped real image

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 262144)         │    26,476,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_4 (Reshape)             │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_8              │ (None, 128, 128, 128)  │       131,200 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_9              │ (None, 256, 256, 64)   │       131,136 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 256, 256, 1)    │           577 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ cropping2d (Cropping2D)         │ (None, 250, 250, 1)    │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 26,739,457 (102.00 MB)

 Trainable params: 26,739,457 (102.00 MB)

 Non-trainable params: 0 (0.00 B)

The Discriminator:

In [ ]:
shape = (250, 250, 1)

# Discriminator's input shape should be the same shape as the generator's output

def build_discriminator(img_shape):
    model = models.Sequential()
    model.add(layers.Conv2D(128, kernel_size=4, strides=2, padding="same", input_shape=img_shape, activation="relu"))
    model.add(layers.Conv2D(256, kernel_size=4, strides=2, padding="same", activation="relu"))
    model.add(layers.Flatten())
    model.add(layers.Dense(1, activation="sigmoid"))
    return model

chemocraft_discriminator = build_discriminator(shape)
chemocraft_discriminator.summary()

Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_13 (Conv2D)              │ (None, 125, 125, 128)  │         2,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 63, 63, 256)    │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 1016064)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │     1,016,065 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,542,785 (5.89 MB)

 Trainable params: 1,542,785 (5.89 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
def compile_gan(generator, discriminator, latent_dim):
    discriminator.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    z = layers.Input(shape=(latent_dim,))
    img = generator(z)
    discriminator.trainable = False
    validity = discriminator(img)
    gan = models.Model(z, validity)
    gan.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    return gan

chemocraft_gan = compile_gan(chemocraft_generator, chemocraft_discriminator,latent_dim=dim)
chemocraft_gan.summary()

Model: "functional_48"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_14 (InputLayer)     │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_6 (Sequential)       │ (None, 250, 250, 1)    │    26,739,457 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_11 (Sequential)      │ (None, 1)              │     1,542,785 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 28,282,242 (107.89 MB)

 Trainable params: 26,739,457 (102.00 MB)

 Non-trainable params: 1,542,785 (5.89 MB)

Connecting Generator & Discriminator through the GAN:

In [23]:
import tensorflow as tf

def train_gan(generator, latent_dim, discriminator, gan, train_slices, epochs, batch_size):
    for epoch in range(epochs):
        for _ in range(len(train_slices) // batch_size):
            # Select random batch of real images
            idx = np.random.randint(0, len(train_slices), batch_size)
            real_slices = np.array([train_slices[i] for i in idx])

            # Generate fake images
            noise = np.random.normal(0, 1, (batch_size, latent_dim))
            fake_slices = generator.predict(noise)

            # Train the discriminator
            d_loss_real = discriminator.train_on_batch(real_slices, np.ones((batch_size, 1)))
            d_loss_fake = discriminator.train_on_batch(fake_slices, np.zeros((batch_size, 1)))
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # Train the generator
            noise = np.random.normal(0, 1, (batch_size, latent_dim))
            g_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))

        print(f"Epoch {epoch + 1}/{epochs}, D Loss: {d_loss[0]}, G Loss: {g_loss[0]}")

Setting up Data Pipeline for Training:

In [ ]:
s3 = boto3.resource('s3')
bucket_name = 'chemocraft-data'
folder_path = 'tanmay/brain_slices/'
bucket = s3.Bucket(bucket_name)

i = 0

for obj in bucket.objects.filter(Prefix=f"{folder_path}002/"):
    i+=1
    # train_gan(chemocraft_generator, dim, chemocraft_discriminator, chemocraft_gan, train_slices=### Modify####, epochs=150, batch_size=5)

print(i)

775


In [7]:
from skimage.metrics import structural_similarity as ssim

def generate_3d_image(generator, latent_dim, num_slices=150):
     noise = np.random.normal(0, 1, (num_slices, latent_dim))
     generated_slices = generator.predict(noise)
     generated_3d_image = np.stack(generated_slices, axis=0)  # Shape: (150, 250, 250)
     return generated_3d_image

def evaluate_3d_image(true_image, generated_image):
       for i in range(true_image.shape[0]):  # For each slice
           slice_ssim = ssim(true_image[i], generated_image[i])
           print(f"SSIM for slice {i}: {slice_ssim}")